In [ ]:
!pip install -q tensorflow xarray netCDF4 pandas scipy scikit-learn xgboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unrar x -o+ /content/drive/MyDrive/Dataset.rar /content/Dataset/


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Dataset.rar

Creating    /content/Dataset                                          OK
Creating    /content/Dataset/Dataset                                  OK
Creating    /content/Dataset/Dataset/daily_mslp                       OK
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1979.nc         0%  1%  2%  OK 
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1980.nc         3%  4%  5%  OK 
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1981.nc         6%  7%  8%  OK 
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1982.nc         9% 10% 11%  OK 
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1983.nc        12% 13%  OK 
Extracting  /content/Dataset/Dataset/daily_mslp/daily_mslp_1984.nc        14% 15% 16%  OK 
Extracting  /

In [ ]:

MSLP_DIR    = "/content/Dataset/Dataset/daily_mslp"
TRACK_DIR   = "/content/Dataset/Dataset/lps_era5_data"

SLP_OUT_DIR  = "/content/unet_data/slp"
MASK_OUT_DIR = "/content/unet_data/mask"
TIME_DIM     = "valid_time"
VAR_NAME     = "msl"

import os
os.makedirs(SLP_OUT_DIR, exist_ok=True)
os.makedirs(MASK_OUT_DIR, exist_ok=True)


In [ ]:

import numpy as np
from scipy.ndimage import minimum_filter, label

def extract_lps_mask(slp_array):
    """
    Returns a binary mask (uint8) where 1 indicates any connected local-minimum zone
    in the input 2D SLP array.
    """

    neigh_min = minimum_filter(slp_array, size=3, mode='mirror')
    base_mask = (slp_array == neigh_min)

    labels, n_zones = label(base_mask)

    return (labels > 0).astype(np.uint8)




In [ ]:

import os
import numpy as np
import xarray as xr


os.makedirs(SLP_OUT_DIR, exist_ok=True)
os.makedirs(MASK_OUT_DIR, exist_ok=True)


import glob
nc_files = sorted(glob.glob(f"{MSLP_DIR}/daily_mslp_*.nc"))
YEARS = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in nc_files]
print("Generating for years:", YEARS)

saved_slp = saved_mask = 0
for yr in YEARS:
    path = f"{MSLP_DIR}/daily_mslp_{yr}.nc"
    with xr.open_dataset(path) as ds:
        for t in ds[TIME_DIM].values:
            date_str = np.datetime_as_string(t, unit='D')
            slp = ds[VAR_NAME].sel({TIME_DIM: t}).values.astype(np.float32)
            mask = extract_lps_mask(slp).astype(np.uint8)

            slp_file  = f"{SLP_OUT_DIR}/slp_{yr}_{date_str}.npy"
            mask_file = f"{MASK_OUT_DIR}/mask_{yr}_{date_str}.npy"

            np.save(slp_file, slp)
            np.save(mask_file, mask)
            saved_slp += 1
            saved_mask += 1

print(f"Saved {saved_slp} SLP files and {saved_mask} mask files to:")
print(" ", SLP_OUT_DIR)
print(" ", MASK_OUT_DIR)


Generating for years: [1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
Saved 4392 SLP files and 4392 mask files to:
  /content/unet_data/slp
  /content/unet_data/mask


In [ ]:

import glob, numpy as np, tensorflow as tf


slp_files  = sorted(glob.glob(f"{SLP_OUT_DIR}/*.npy"))
mask_files = sorted(glob.glob(f"{MASK_OUT_DIR}/*.npy"))


print("Found", len(slp_files), "SLP .npy files and", len(mask_files), "mask .npy files.")
if len(slp_files) == 0 or len(mask_files) == 0:
    raise RuntimeError(
        "No .npy files found! Make sure you have run the generation cell (Cell N+3) "
        "to produce the SLP & mask .npy pairs before this step."
    )
assert len(slp_files) == len(mask_files), "Mismatch between SLP and mask files"


def load_pair(path_slp, path_mask):
    slp_file  = path_slp.numpy().decode('utf-8')
    mask_file = path_mask.numpy().decode('utf-8')
    slp  = np.load(slp_file).astype(np.float32)
    mask = np.load(mask_file).astype(np.uint8)
    slp  = ((slp - slp.mean()) / slp.std())[..., np.newaxis]  # (241,361,1)
    mask = mask[..., np.newaxis]                               # (241,361,1)
    return slp, mask


dataset = tf.data.Dataset.from_tensor_slices((slp_files, mask_files))
dataset = dataset.map(lambda x,y: tf.py_function(load_pair, [x,y], [tf.float32, tf.uint8]),
                      num_parallel_calls=tf.data.AUTOTUNE)


def fix_shapes(slp, mask):
    slp  = tf.ensure_shape(slp,  [241,361,1])
    mask = mask[:240, :360, :]
    mask = tf.ensure_shape(mask, [240,360,1])
    return slp, mask

dataset = dataset.map(fix_shapes, num_parallel_calls=tf.data.AUTOTUNE)


total  = len(slp_files)
n_train = int(0.7 * total)
n_val   = int(0.15 * total)
n_test  = total - n_train - n_val


dataset = dataset.shuffle(buffer_size=total, seed=42)

train_ds = dataset.take(n_train).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds   = dataset.skip(n_train).take(n_val).batch(8).prefetch(tf.data.AUTOTUNE)
test_ds  = dataset.skip(n_train + n_val).batch(8).prefetch(tf.data.AUTOTUNE)

print(f"Splits → train: {n_train}, val: {n_val}, test: {n_test}")


Found 4392 SLP .npy files and 4392 mask .npy files.
Splits → train: 3074, val: 658, test: 660


In [ ]:

from tensorflow.keras import layers, Model

def build_unet_dropout(input_shape=(241,361,1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32,3,activation='relu',padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Dropout(0.1)(c1)
    c1 = layers.Conv2D(32,3,activation='relu',padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(64,3,activation='relu',padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Dropout(0.1)(c2)
    c2 = layers.Conv2D(64,3,activation='relu',padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(128,3,activation='relu',padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Dropout(0.2)(c3)
    c3 = layers.Conv2D(128,3,activation='relu',padding='same')(c3)

    u4 = layers.Conv2DTranspose(64,2,strides=2,padding='same')(c3)
    u4 = layers.concatenate([u4, c2])
    c4 = layers.Conv2D(64,3,activation='relu',padding='same')(u4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Dropout(0.1)(c4)
    c4 = layers.Conv2D(64,3,activation='relu',padding='same')(c4)

    u5 = layers.Conv2DTranspose(32,2,strides=2,padding='same')(c4)
    c1_crop = layers.Cropping2D(cropping=((0,1),(0,1)))(c1)
    u5 = layers.concatenate([u5, c1_crop])
    c5 = layers.Conv2D(32,3,activation='relu',padding='same')(u5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Dropout(0.1)(c5)
    c5 = layers.Conv2D(32,3,activation='relu',padding='same')(c5)

    outputs = layers.Conv2D(1,1,activation='sigmoid')(c5)
    return Model(inputs, outputs)

unet = build_unet_dropout((241,361,1))    # or build_unet(...) if you’re using that
unet.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.MeanIoU(num_classes=2)]
)
unet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 241, 361,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 241, 361,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 241, 361,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 241, 361,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 241, 361,  │      9,248 │ dropout[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 120, 180,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 120, 180,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 180,  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 120, 180,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 120, 180,  │     36,928 │ dropout_1[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 60, 90,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 60, 90,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 90,    │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 90,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 60, 90,    │    147,584 │ dropout_2[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 120, 180,  │     32,832 │ conv2d_5[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 120, 180,  │          0 │ conv2d_transpose

 Total params: 467,233 (1.78 MB)

 Trainable params: 466,593 (1.78 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_unet.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

history = unet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=callbacks
)

Epoch 1/3
 39/385 ━━━━━━━━━━━━━━━━━━━━ 1:22:39 14s/step - loss: 1.0096 - mean_io_u: 0.4949

In [ ]:

import numpy as np
from sklearn.metrics import precision_recall_curve

y_val_true = np.concatenate([y.numpy().flatten() for x,y in val_ds], axis=0)
y_val_prob = unet.predict(val_ds).flatten()

prec, rec, thresh = precision_recall_curve(y_val_true, y_val_prob)
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-8)
best_idx = np.argmax(f1_scores)
best_thresh = thresh[best_idx]
print(f"Best threshold: {best_thresh:.3f}, F1: {f1_scores[best_idx]:.3f}")

In [ ]:

from sklearn.metrics import accuracy_score, f1_score, jaccard_score

unet.load_weights('best_unet.h5')

y_test_true = np.concatenate([y.numpy().flatten() for x,y in test_ds], axis=0)
y_test_prob = unet.predict(test_ds).flatten()
y_test_pred = (y_test_prob >= best_thresh).astype(np.uint8)

print("Test Pixel Accuracy:", accuracy_score(y_test_true, y_test_pred))
print("Test F1-score:", f1_score(y_test_true, y_test_pred))
print("Test IoU:", jaccard_score(y_test_true, y_test_pred))